In [2]:
import tensorflow as tf
import torch

print("TensorFlow version:", tf.__version__)
print("PyTorch version:", torch.__version__)


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# 데이터 증강 및 전처리 설정
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # 모든 픽셀 값을 0~1로 스케일링
    rotation_range=20,  # 회전 증강
    width_shift_range=0.2,  # 가로 이동 증강
    height_shift_range=0.2,  # 세로 이동 증강
    shear_range=0.2,  # 전단 변환
    zoom_range=0.2,  # 확대/축소
    horizontal_flip=True  # 수평 뒤집기
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255  # 검증 데이터는 스케일링만 적용
)

# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # EfficientNetV2의 입력 크기
    batch_size=32,
    class_mode='categorical'  # 다중 클래스 분류
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# EfficientNetV2 모델 불러오기
base_model = EfficientNetV2L(
    weights='imagenet',  # ImageNet 사전 학습 가중치
    include_top=False,
    input_shape=(224, 224, 3)  # 입력 이미지 크기
)
base_model.trainable = False  # 사전 학습된 가중치를 고정

# 모델 구성
from tensorflow.keras import layers, models
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # 클래스 개수
])

# 모델 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백 설정
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

# 모델 훈련
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)

# 훈련 완료 후 평가
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")





NameError: name 'ImageDataGenerator' is not defined